In [7]:
%env BASE_DIR=/home/jupyter/project5

env: BASE_DIR=/home/jupyter/project5


In [8]:
!gsutil cp gs://cs327e-open-access/fs_shopify.zip $BASE_DIR

Copying gs://cs327e-open-access/fs_shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [9]:
!gsutil cp gs://cs327e-open-access/fs_samples.zip $BASE_DIR

Copying gs://cs327e-open-access/fs_samples.zip...
/ [1 files][  4.5 KiB/  4.5 KiB]                                                
Operation completed over 1 objects/4.5 KiB.                                      


In [10]:
!unzip $BASE_DIR/fs_shopify.zip

Archive:  /home/jupyter/project5/fs_shopify.zip
   creating: fs_shopify/
  inflating: fs_shopify/apps.csv     
  inflating: fs_shopify/reviews.csv  
  inflating: fs_shopify/apps_categories.csv  
  inflating: fs_shopify/key_benefits.csv  
  inflating: fs_shopify/categories.csv  
  inflating: fs_shopify/pricing_plans.csv  
  inflating: fs_shopify/pricing_plan_features.csv  


In [11]:
!unzip $BASE_DIR/fs_samples.zip

Archive:  /home/jupyter/project5/fs_samples.zip
   creating: fs_samples/
  inflating: fs_samples/delete_single.py  
  inflating: fs_samples/write_batch.py  
  inflating: fs_samples/read_nested.py  
  inflating: fs_samples/read_count.py  
  inflating: fs_samples/write_single.py  
  inflating: fs_samples/read_where.py  
  inflating: fs_samples/write_nested.py  
  inflating: fs_samples/delete_field.py  
  inflating: fs_samples/read_single.py  
  inflating: fs_samples/update_single.py  
  inflating: fs_samples/read_doc_with_filter2.py  


In [ ]:
import pandas as pd
from google.cloud import firestore

db = firestore.Client()

# read csv files
df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
reviews_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
key_benefits_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pricing_plans_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
pricing_plan_features_rows = df.values.tolist()

def batch_commit(apps_rows):
    batch = db.batch()
    for apps_row in apps_rows: 
        apps_record = {}
        apps_record['id'] = apps_row[0]
        apps_record['url'] = apps_row[1]
        apps_record['title'] = apps_row[2]
        apps_record['developer'] = apps_row[3]
        apps_record['developer_link'] = apps_row[4]
        apps_record['icon'] = apps_row[5]
        apps_record['rating'] = apps_row[6]
        apps_record['reviews_count'] = apps_row[7]

        apps_ref = db.collection('apps').document(apps_row[0])
        batch.set(apps_ref, apps_record)

        for reviews_row in reviews_rows:
            if apps_row[0] == reviews_row[0]:
                reviews_record = {}
                reviews_record['author'] = reviews_row[1]
                reviews_record['rating'] = reviews_row[2]
                reviews_record['posted_at'] = reviews_row[3]
                if reviews_row[1] == ".":
                    reviews_row[1] = "dot"
                reviews_ref = apps_ref.collection('reviews').document(str(reviews_row[1]).replace('/', ' '))

                batch.set(reviews_ref, reviews_record)

        for key_benefits_row in key_benefits_rows:
            if apps_row[0] == key_benefits_row[0]:
                key_benefits_record = {}
                key_benefits_record['title'] = key_benefits_row[1]
                key_benefits_record['description'] = key_benefits_row[2]

                key_benefits_ref = apps_ref.collection('key_benefits').document(key_benefits_row[1].replace('/', ' '))

                batch.set(key_benefits_ref, key_benefits_record)

        for pricing_plans_row in pricing_plans_rows:
            if apps_row[0] == pricing_plans_row[1]:
                pricing_plans_record = {}
                pricing_plans_record['id'] = pricing_plans_row[0]
                pricing_plans_record['title'] = pricing_plans_row[2]
                pricing_plans_record['price'] = pricing_plans_row[3]
                pricing_plans_record['feature'] = []
                for pricing_plan_features_row in pricing_plan_features_rows:
                    if apps_row[0] == pricing_plan_features_row[0] and pricing_plans_row[0] == pricing_plan_features_row[1]:
                        pricing_plans_record['feature'].append(pricing_plan_features_row[2])


                pricing_plans_ref = apps_ref.collection('pricing_plans').document(pricing_plans_row[0])

                batch.set(pricing_plans_ref, pricing_plans_record)
           
    batch.commit()
    
# Split rows into mini-batches of size 500
chunk_size = 50
for i in range(0, len(apps_rows), chunk_size):
    print(i)
    chunk = apps_rows[i:i+chunk_size]
    batch_commit(chunk)

In [ ]:
import pandas as pd
from google.cloud import firestore

db = firestore.Client()

# read csv files
df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/categories.csv', sep=',', header=0, lineterminator='\n')
categories_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
apps_categories_rows = df.values.tolist()

batch = db.batch()
for categories_row in categories_rows: 
    categories_record = {}
    categories_record['id'] = categories_row[0]
    categories_record['title'] = categories_row[1]

    categories_ref = db.collection('categories').document(categories_row[0])
    batch.set(categories_ref, categories_record)

    for apps_categories_row in apps_categories_rows:
        if apps_categories_row[1] == categories_row[0]:
            for apps_row in apps_rows: 
                if apps_categories_row[0] == apps_row[0]:
                    apps_record = {}
                    apps_record['id'] = apps_row[0]
                    apps_record['url'] = apps_row[1]
                    apps_record['title'] = apps_row[2]
                    apps_record['developer'] = apps_row[3]
                    apps_record['developer_link'] = apps_row[4]
                    apps_record['icon'] = apps_row[5]
                    apps_record['rating'] = apps_row[6]
                    apps_record['reviews_count'] = apps_row[7]

                    apps_ref = categories_ref.collection('apps').document(apps_row[0])
                    batch.set(apps_ref, apps_record)   
batch.commit()

### Retrieve the number of documents in each collection and subcollection

#### Apps Collection

In [ ]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter
from google.cloud.firestore_v1 import aggregation

apps_ref = db.collection('apps')
apps_count = apps_ref.count().get()
print('Count of apps collection:', apps_count[0][0].value)

results = apps_ref.stream()

for result in results:
    print('App ' + result.id + ':')
    reviews_ref = apps_ref.document(result.id).collection('reviews')
    reviews_result = reviews_ref.count().get()
    print('    Count of reviews subcollection:', reviews_result[0][0].value )
    
    key_benefits_ref = apps_ref.document(result.id).collection('key_benefits')
    key_benefits_result = key_benefits_ref.count().get()
    print('    Count of key_benefits subcollection', key_benefits_result[0][0].value)

    pricing_plans_ref = apps_ref.document(result.id).collection('pricing_plans')
    pricing_plans_result = pricing_plans_ref.count().get()
    print('    Count of pricing_plans subcollection:', pricing_plans_result[0][0].value)

#### Categories Collection

In [27]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter
from google.cloud.firestore_v1 import aggregation

categories_ref = db.collection('categories')
categories_count = categories_ref.count().get()
print('Count in categories collection:', categories_count[0][0].value)

results = categories_ref.stream()

for result in results:
    print('Category ' + result.id + ':')
    apps_ref = categories_ref.document(result.id).collection('apps')
    apps_result = apps_ref.count().get()
    print('    Count in apps subcollection:', apps_result[0][0].value)

Count in categories collection: 12
Category 26a72de0d02e0e4e5f615332d61a878e:
    Count in apps subcollection: 133
Category 30a930262efca129caafcd586bc7e6fe:
    Count in apps subcollection: 83
Category 30ea2315e910c5e5335de42d5e87dca5:
    Count in apps subcollection: 602
Category 5eb4e29e50e3f178acc614236ed107f4:
    Count in apps subcollection: 957
Category 64430ad2835be8ad60c59e7d44e4b0b1:
    Count in apps subcollection: 310
Category 737ad50051083aa051d127a53b3ac0da:
    Count in apps subcollection: 190
Category 9fe78acd078fc030de72c896adc93a2d:
    Count in apps subcollection: 251
Category bd125b49ce6b0425d29e33cb8b49a496:
    Count in apps subcollection: 176
Category c3f9db73c29bc33607778f9935c3c4dc:
    Count in apps subcollection: 1537
Category c576a841fd4f333a6f074d68e76a1d37:
    Count in apps subcollection: 249
Category c769c2bd15500dd906102d9be97fdceb:
    Count in apps subcollection: 794
Category e44cd5379a3166568741a07cc81ef6e5:
    Count in apps subcollection: 101


#### List the top 10 "Productivity" apps (whose categories.title = "Productivity") sorted by their rating in descending order. Return the id, title, developer, rating and reviews_count for those apps. Limit the results to the first 10 records.

In [26]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter

db = firestore.Client()
categories_ref = db.collection('categories')
query = categories_ref.where(filter=FieldFilter('title', '==', 'Productivity'))
results = query.stream()
categories_ref = db.collection('categories').document(next(results).to_dict()['id']).collection('apps')
query = categories_ref.order_by('rating', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()

print('{:<36}  {:<30}  {:<21}  {:<6}  {:<3}'.format('id', 'title', 'developer', 'rating', 'reviews_count'))
for doc in results:
    doc = doc.to_dict()
    print('{:<36}  {:<30}  {:<21}  {:<6}  {:<3}'.format(doc['id'], doc['title'], doc['developer'], doc['rating'], doc['reviews_count']))

id                                    title                           developer              rating  reviews_count
fe5ae45c-379c-42bf-be7d-b5f6e15fc13c  Customer Tags                   Union Works Apps       5.0     21 
f99bb1e3-f326-4f10-8901-491652e9809b  Order Tagger                    Union Works Apps       5.0     68 
f864e3bd-da0e-41dc-be65-984325331475  SilkRoad ‑ Facebook Auto Ads    SilkRoad               5.0     2  
f5344e64-9cda-4d97-b198-2aaeb5170518  Xporter Data Export Tool        Modd Apps Inc.         5.0     223
ed77a32d-0fa3-458b-b639-e01ea7b78ec0  FraudBlock Fraud Prevention     ShopFox                5.0     6  
e0e231d6-4988-4a8c-ad84-cf18d0f38738  Automation Fox                  Automation Fox         5.0     1  
d75fa395-3a4d-41c3-88d1-2f802c1e2411  AirPower                        BaseGenius             5.0     1  
d0be03ff-f74d-4fb5-8d2c-79541a1aea1b  Excelify                        Excelify.io            5.0     178
c9a86f8c-817e-4bb5-bbdd-e0e669bd7816  UPC Cod

#### List the 10 apps with the highest number of reviews (based on apps.review_count). Return the id, title, developer, rating and reviews_count for those apps. Order the results by reviews_count in descending order.

In [8]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter

db = firestore.Client()
apps_ref = db.collection('apps')
query = apps_ref.order_by('reviews_count', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()

print('{:<36}  {:<30}  {:<21}  {:<6}  {:<3}'.format('id', 'title', 'developer', 'rating', 'reviews_count'))
for doc in results:
    doc = doc.to_dict()
    print('{:<36}  {:<30}  {:<21}  {:<6}  {:<3}'.format(doc['id'], doc['title'], doc['developer'], doc['rating'], doc['reviews_count']))

id                                    title                           developer              rating  reviews_count
d9f142ee-b141-4dc4-9353-173db61d2eb0  Privy ‑ Exit Pop Ups & Email    Privy                  4.7     23078
78ea0810-c008-4a4e-a82f-de0c790e3286  Free Shipping Bar               Hextom                 4.9     8737
b88488b0-9912-44d3-b736-224c36f09d95  Sales Pop ‑ Popup Notification  CartKit                4.8     6905
e528a60e-94f8-4e92-80e2-5bc6013b8283  BEST Currency Converter         Grizzly Apps           4.8     5986
be2640c4-01b5-4d52-9f68-cae8c0734d0d  Recart FB Messenger Marketing   Recart                 4.8     5596
70bff9e0-4316-4cc6-84ce-92fcd1bc6925  EU Cookie Bar ‑ Cookie GDPR     Booster Apps           4.7     5259
171816e2-27d4-4552-a65e-ab44a312fe04  Sales Pop Master ‑ Countdown    Autoketing             4.8     4931
9025eff0-d714-4df1-930f-43f5582979ad  Ultimate Sales Boost            Hextom                 4.8     4847
404183ac-6d52-4960-85d2-8350f37c2f0b